<a href="https://colab.research.google.com/github/gustavo-ifusp/prefeitura_smit_sp360/blob/main/002%20-%20DAG_passageiros_transportados_automatiza%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Passareiros Transportados SPTRANS**
**Link dos Dados de origem:**
https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932<br>
<br>**Notebook da leitura retroativa:**<br>
https://colab.research.google.com/drive/1R_gGCuGN-pZbpFbyH0Bv5A_ZNv3yenur?usp=sharing
<br>**Notebook de automação da leitura:**<br>
https://colab.research.google.com/drive/1KbqjyNzNn4J9JvsQO-HthIg7cnJI3ePJ?usp=sharing
<br>**Arquivos:**<br>
https://drive.google.com/drive/folders/16F_gT2tB9lF0ToIh1ftqKnbtrwHrty4X?usp=sharing

<br>**Planilhas prontas:**<br>
https://drive.google.com/file/d/1GS5DIIp2o1nL3o4guXATG0cC0vUdNeLg/view?usp=sharing

<br>**Por ano:**<br>
https://drive.google.com/file/d/1rh9w24382SKv_2qH2F0-qJxBs6ZdKfSV/view?usp=sharing
https://drive.google.com/file/d/1azjjQsAVRtQqqdiYAGWNxNMWnAesulen/view?usp=sharing
https://drive.google.com/file/d/1eAqgEP9gDS6SP5ZO4_ybg8Gug69UsBnI/view?usp=sharing

<br>**Por Mês:**<br>
https://drive.google.com/drive/folders/115-tVXZYdL82MfoUpxKAIc7zlzAJfF9q?usp=sharing

#**Automatização da leitura:**
https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932

Só executará no ambiente correto de execução, no Airflow!

Projeto Final - última versão

In [ ]:
############################################################################################################################################################
# -*- coding: utf-8 -*-
# --------------------------------------------------------------------------------------------------------------------
# Projeto:      SP360
# Órgão:        SPTRANS
# Assunto:      Passageiros Transportados / Demanda de Passageiros
# Conteúdo:     Controle da soma dos passageiros tranpostados por tipo de passagem diariamente e estatística descritiva por dia da semana. 
# Responsável:  Gustavo de Andrade (gustavo.andrade@prefeitura.sp.gov.br)
# Orientação:   Murillo Corvino Rocha    
# Repositório:  https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932
# Descrição:
# Essa DAG é responsável por importar os dados e realizar a somatoria diariamente dos passageiros transportados da SPtrans, verificando e notificando atrasos de publicar os dados no repositorio.
# O Segundo objetivo é trazer uma 'query' dos dados salvos para realizar uma estatística descritiva, salvando em cima toda vez que executar.  
# Caso não haja atualização será gerado um erro, e caso ultrapasse 6 dias sem atualização gerará uma notificação no MSTeams para toda equipe de dados. 
# Schedule:
# Todos os dias programado as 4 horas
# Os dados disponibilizados pela Secretaria de Transporte são de atualização diariamente.
#
# Variáveis do Airflow:
#
# - AMBIENTE: apenas um descritivo do ambiente (HOMOLOGAÇÃO/PRODUÇÃO) para fins de notificações (se estiverem ligadas).
# - PASSAGEIROS_NOTIFICACOES: liga (True) ou desliga (False) as notificações via Teams. Padrão: False
# - PASSAGEIROS_WEBHOOK: URL para o webhook do MS Teams para fins de notificações (se estiverem ligadas).
#
############################################################################################################################################################
import os
import pandas as pd
import numpy as np
import bs4
import requests
from builtins import range
import re
import airflow
from airflow.models import Variable
from airflow.models import DAG
from airflow.exceptions import AirflowSkipException
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from airflow.providers.mysql.hooks.mysql import MySqlHook
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from datetime import datetime, date, timedelta
from sqlalchemy.engine.base import Connection
from bs4 import BeautifulSoup, SoupStrainer

# --------------------------------------------------------------------------------------------------------------------
# VARIÁVEIS GLOBAIS
# --------------------------------------------------------------------------------------------------------------------

CONNECTION_ID  = "..."
nome_dado = "TRANSPORTE_PASSAGEIROS_SPTRANS"
DAG_TAGS  = ['SPTRANS']

# --------------------------------------------------------------------------------------------------------------------
# Funções utilizadas pela DAG
# --------------------------------------------------------------------------------------------------------------------

def parse_dados(pagina_html, nome_mes, dia):
    soup = BeautifulSoup(pagina_html, 'html.parser')
    calendario_mes = soup.find("caption", text = nome_mes).parent
    elemento_link_dia = calendario_mes.find("a", text = dia)
    if elemento_link_dia is None:
        df = pd.DataFrame()
        return df, None
    else:   
        url_do_dia = elemento_link_dia['href']
        passageiros_transportados = requests.get(url_do_dia)
        df = pd.read_excel(passageiros_transportados.content, skiprows = range(0, 2))
        return df, url_do_dia

def obter_conexao_db() -> Connection:
    hook = MySqlHook(mysql_conn_id='...(segredinho)')
    uri = hook.get_uri()
    return create_engine(uri).connect()

def converteNumMesNome(numMes):
    dicionario_meses = {
           1  : ['Janeiro','JAN'],
           2  : ['Fevereiro','FEV'],
           3  : ['Março','MAR'],
           4  : ['Abril','ABR'],
           5  : ['Maio','MAI'],
           6  : ['Junho','JUN'],
           7  : ['Julho','JUL'],
           8  : ['Agosto','AGO'],
           9  : ['Setembro','SET'],
           10 : ['Outubro','OUT'],
           11 : ['Novembro','NOV'],
           12 : ['Dezembro','DEZ']
    }
    return dicionario_meses[numMes]

def converteNumDiaSem(numDiaSem):
    dicionario_dia_semana = {
           0  : 'Segunda-feira',
           1  : 'Terça-feira',
           2  : 'Quarta-feira',
           3  : 'Quinta-feira',
           4  : 'Sexta-feira',
           5  : 'Sábado',
           6  : 'Domingo'
    }
    return dicionario_dia_semana[numDiaSem] 

def atualizar_metadados(nome_dado, nome_arquivo, ultima_data_salva, ultima_data_referencia):
    my_hook = MySqlHook(mysql_conn_id='segredinho')
    url = my_hook.get_uri()
    my_conn = create_engine(url).connect()
    query ="CALL segredinho.segredinho('{nome_dado}​​​​​​​​', '{nome_arquivo}​​​​​​​​', '{ultima_data_salva}​​​​​​​​', '{ultima_data_referencia}​​​​​​​​');".format(nome_dado=nome_dado, nome_arquivo=nome_arquivo, ultima_data_salva=ultima_data_salva, ultima_data_referencia=ultima_data_referencia)
    my_conn.execute(text(query).execution_options(autocommit=True))
    my_conn.close()

def contabilizar_dados_transporte(df, data_atual):
        df.rename(columns={'Data':'data_contabilizacao'}, inplace = True)
        total_passagem_dia = df.groupby(['data_contabilizacao'])['Tot Passageiros Transportados'].sum()
        passageiros_pag_dinheiro = df.groupby(['data_contabilizacao'])['Passageiros Pagtes Em Dinheiro'].sum()
        passageiros_comum_vt = df.groupby(['data_contabilizacao'])['Passageiros Comum e VT'].sum()
        passageiros_pag_bu_comum = df.groupby(['data_contabilizacao'])['Passageiros Pgts Bu Comum M'].sum()
        passageiros_pag_estudantes = df.groupby(['data_contabilizacao'])['Passageiros Pagtes Estudante'].sum()
        passageiros_pag_bu_est_mensal = df.groupby(['data_contabilizacao'])['Passageiros Pgts Bu Est Mensal'].sum()
        passageiros_pag_bu_vt_mensal = df.groupby(['data_contabilizacao'])['Passageiros Pgts Bu Vt Mensal'].sum()
        passageiros_pag = df.groupby(['data_contabilizacao'])['Passageiros Pagantes'].sum()
        passageiros_int_onibus_onibus = df.groupby(['data_contabilizacao'])['Passageiros Int Ônibus->Ônibus'].sum()
        passageiros_gratuidade = df.groupby(['data_contabilizacao'])['Passageiros Com Gratuidade'].sum()
        passageiros_gratuidade_est = df.groupby(['data_contabilizacao'])['Passageiros Com Gratuidade Est'].sum()
        passageiros_tranportados_sptrans = pd.concat([total_passagem_dia, passageiros_pag_dinheiro, passageiros_comum_vt, passageiros_pag_bu_comum,
        passageiros_pag_estudantes, passageiros_pag_bu_est_mensal, passageiros_pag_bu_vt_mensal, passageiros_pag, 
        passageiros_int_onibus_onibus, passageiros_gratuidade, passageiros_gratuidade_est],axis = 1)
        passageiros_tranportados_sptrans.rename(columns={
                                                        'Tot Passageiros Transportados':'total_passageiros_transportados',
                                                        'Passageiros Pagtes Em Dinheiro':'passageiros_pagtes_em_dinheiro',
                                                        'Passageiros Comum e VT':'passageiros_comum_e_vt',
                                                        'Passageiros Pgts Bu Comum M':'passageiros_pgts_bu_comum_m',
                                                        'Passageiros Pagtes Estudante':'passageiros_pagtes_estudante',
                                                        'Passageiros Pgts Bu Est Mensal':'passageiros_pgts_bu_est_mensal',
                                                        'Passageiros Pgts Bu Vt Mensal':'passageiros_pgts_bu_vt_mensal',
                                                        'Passageiros Pagantes':'passageiros_pagantes',
                                                        'Passageiros Int Ônibus->Ônibus':'passageiros_int_onibus_onibus',
                                                        'Passageiros Com Gratuidade':'passageiros_com_gratuidade',
                                                        'Passageiros Com Gratuidade Est':'passageiros_com_gratuidade_est',
                                                        }, inplace = True)
        numDiaSem = data_atual.weekday()
        Dia_da_semana = converteNumDiaSem(numDiaSem)
        passageiros_tranportados_sptrans['dia_da_semana'] = Dia_da_semana
        df = passageiros_tranportados_sptrans
        return df
        
        
def notificar_msteams(mensagem: str, webhook: str):
    print("teste de linha 1: notif_msteamns")
    headers = {"Content-type": "application/json"}
    text = "```{}```".format(mensagem)
    data = {"text": text}
    requests.post(webhook, headers=headers, json=data)

def on_failure(dias_sem_atualizar):
    enviar_notificacoes = Variable.get("PASSAGEIROS_NOTIFICACOES", default_var="False")
    if enviar_notificacoes != 'True':
        print("O envio de notificações está desligado.")
    else:
        webhook = Variable.get("PASSAGEIROS_WEBHOOK")
        mensagem = "Erro ao executar a DAG de Passageiros Transportados: 'sptrans_etl_testeTransportesPassageiros'. Sem atualização a " + str(dias_sem_atualizar) + " dia(s)."
        notificar_msteams(mensagem, webhook)
        raise Exception("Erro ao executar a dag de passageiros transportados. Não existe atualização nos últimos a mais de 7 dias.")

def recuperar_dados_url(url_arquivo):
    nome_arquivo = re.split(r'https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/', url_arquivo)
    nome_arquivo = nome_arquivo[1]
    return nome_arquivo

def carregar_dados_transporte(ds, **kwargs): # kwargs é Keyword Arguments (parâmetros nomeados). Para número indefinido de parâmetros, sem nomeá-los, utilizaria apenas um asterisco.
    my_conn = obter_conexao_db()
    
    getUltimaDataCarga = """
                          SELECT data_contabilizacao 
                            FROM fato_total_passageiros_transportados_sptrans
                        ORDER BY data_contabilizacao DESC 
                           LIMIT 1
                         """

    data_ultima_carga = my_conn.execute(getUltimaDataCarga).first()[0]    
    data_atual = data_ultima_carga 
    data_atual = data_atual + timedelta(days=1)  
    data_hoje = date.today()
    all_data = pd.DataFrame()
    url_arquivo = None
    ultimo_df = None    
    while data_atual < data_hoje:
            mes = data_atual.month
            dia = data_atual.day
            dias_sem_atualizar = (data_hoje - data_ultima_carga).days
            req = requests.get('https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932')
            df, url_temp = parse_dados(req.content, converteNumMesNome(mes)[0], dia)
            if url_temp != None:
                url_arquivo = url_temp
            if df.empty:
                break
            ultimo_df = contabilizar_dados_transporte(df, data_atual)
            all_data = all_data.append(ultimo_df)
            data_atual = data_atual + timedelta(days=1) 
    dias_sem_atualizar = (data_hoje - data_ultima_carga).days
    if all_data.empty:
        if dias_sem_atualizar > 6:
            on_failure(dias_sem_atualizar)
        else:
            print("Sem atualização: ", dias_sem_atualizar," / 7 dias")
        raise AirflowSkipException # raise tem como finalidade invocar uma Exception no momento oportuno, ou seja invocar o erro! 
    all_data.to_sql('fato_total_passageiros_transportados_sptrans', my_conn, if_exists='append' , method='multi')
    nome_arquivo = recuperar_dados_url(url_arquivo)
    ultima_data_salva = my_conn.execute(getUltimaDataCarga).first()[0]
    ultima_data_salva = ultima_data_salva.strftime('%Y-%m-%d')  
    ultima_data_referencia = ultima_data_salva

    #Aqui começa a Estatística Descritiva, onde readiquirimos os dados para fazer a média e mediana e salvar em uma nova tabela com os dados modelados. 

    getUltimosDadosSalvos = """
                            SELECT data_contabilizacao, dia_da_semana, total_passageiros_transportados FROM fato_total_passageiros_transportados_sptrans;
                            """   
    all_data_completo = pd.read_sql(getUltimosDadosSalvos, my_conn) 
    all_data_completo['data_contabilizacao'] = pd.to_datetime(all_data_completo['data_contabilizacao'])
    all_data_completo['dia_da_semana'] = all_data_completo['dia_da_semana'].astype("category")
    all_data_completo['mes'] = all_data_completo['data_contabilizacao'].dt.month
    dados_media = all_data_completo.groupby(['dia_da_semana', 'mes'], as_index=False)['total_passageiros_transportados'].mean()
    dados_mediana = all_data_completo.groupby(['dia_da_semana', 'mes'], as_index=False)['total_passageiros_transportados'].median()
    dados_media = dados_media.rename(columns={'total_passageiros_transportados': 'media_passageiros_transportados'})
    dados_mediana =  dados_mediana.rename(columns={'total_passageiros_transportados': 'mediana_passageiros_transportados'})
    dados_mediana = dados_mediana.drop(['dia_da_semana','mes'], axis = 1)
    estat_descritiva = pd.concat([dados_media, dados_mediana],axis = 1)
    estat_descritiva.to_sql('fato_total_passageiros_transportados_sptrans_estatistica_por_mes', my_conn,  index = False, if_exists='replace' , method='multi')

    atualizar_metadados(nome_dado, nome_arquivo, ultima_data_salva, ultima_data_referencia) 

args = {
    'owner': 'airflow',
    'start_date': airflow.utils.dates.days_ago(1),
}

dag = DAG(
    dag_id='...',
    default_args=args,
    schedule_interval='0 4 * * *',  # define a periodicidade de execução
    dagrun_timeout=timedelta(minutes=240)  # tempo de execuçao permitida para a DAG antes de interrompela
)

carregar = PythonOperator(
    task_id='...',
    provide_context=True,
    python_callable=carregar_dados_transporte,
    dag=dag)

carregar

#**Ambiente de Testes**


In [ ]:
from bs4 import BeautifulSoup

url = "https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932"
soup = BeautifulSoup("<html>url</html>")
soup = BeautifulSoup('html.parser')
soup.find_all('a')
for link in soup.find_all('a'):
    soup = BeautifulSoup('<p class="calend_esq"></p>')
    soup.p['class']
soup.p.get_attribute_list('calend_esq')

[None]

In [ ]:
soup

<html><body><p>html.parser</p></body></html>

In [ ]:
r = requests.get('https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932')
r.status_code

200

In [ ]:
r = requests.get('https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932')
r.status_code

200

In [ ]:
r.headers['content-type']

'text/html; charset=UTF-8'

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')
soup

<!DOCTYPE doctype html>
 <html lang="pt-br"> <head> <meta charset="utf-8"/> <meta content="ie=edge" http-equiv="x-ua-compatible"/> <meta content="width=device-width, initial-scale=1" name="viewport"/> <meta content="" name="copyright"> <meta content="general" name="rating"> <meta content="Prefeitura da Cidade de São Paulo" name="DC.publisher"> <meta content="0 days" name="revisit-after"/> <meta content="#eee" name="theme-color"/> <meta content="Passageiros Transportados - 2021" name="title"> <meta content="" name="description"/> <meta content="2021-01-06" name="DC.date.created"/> <meta content="2021-01-06" name="DC.date.modified"/> <meta content="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/comunicacao/prefeitura_sp_200x200.jpg" property="og:image"/> <title>Passageiros Transportados - 2021 | Secretaria Municipal de Mobilidade e Transportes | Prefeitura da Cidade de São Paulo</title> <link href="https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/

In [ ]:
soup2 = soup.find('div', class_ = 'calendarios')
soup2

<div class="calendarios">
<div class="calend_esq">
<table>
<caption>Janeiro</caption>
<thead>
<tr>
<th>Dom</th>
<th>Seg</th>
<th>Ter</th>
<th>Qua</th>
<th>Qui</th>
<th>Sex</th>
<th>Sáb</th>
</tr>
</thead>
<tbody>
<tr>
<td colspan="5"> </td>
<td><a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/01JAN2021.xls">1</a></td>
<td><a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/02JAN2021.xls">2</a></td>
</tr>
<tr>
<td><a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/03JAN2021.xls">3</a></td>
<td><a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/04JAN2021.xls">4</a></td>
<td><a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/05JAN2021.xls">5</a></td>
<td><a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/06JAN2021.xls">6</a></td>
<td><a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/07JAN2021.xls">7</a></td>
<td><a href="https://www.prefeitura.sp.gov.br/cidade/secretari

In [ ]:
data_atual = date.today()
ano = data_atual.year
m = data_atual.month
d = data_atual.day
anos = str(ano)
d -= 1
print("Ontem foi:", d,"/", m,"/", ano, "- Primeira data a se buscar novo registro!")

Ontem foi: 28 / 6 / 2021 - Primeira data a se buscar novo registro!


In [ ]:
url0 = "https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/"
if m == 12:
    mes = "DEZ"
elif m == 11:
    mes = "NOV"
elif m == 10:
    mes = "OUT"
elif m == 9:
    mes = "SET"
elif m == 8:
    mes = "AGO"
elif m == 7:
    mes = "JUL"
elif m == 6:
    mes = "JUN"
elif m == 5:
    mes = "MAI"
elif m == 4:
    mes = "ABR"
elif m == 3:
    mes = "MAR"
elif m == 2:
    mes = "FEV"
elif m == 1:
    mes = "JAN"           
mes

'JUN'

In [ ]:
dia = str(d)+mes+anos
dia

'28JUN2021'

In [ ]:
print("Ontem foi:", d,mes,ano, "- No formato dos linkis")

Ontem foi: 28 JUN 2021 - No formato dos linkis


In [ ]:
list_links = soup2.find_all('a')
list_links

[<a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/01JAN2021.xls">1</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/02JAN2021.xls">2</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/03JAN2021.xls">3</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/04JAN2021.xls">4</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/05JAN2021.xls">5</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/06JAN2021.xls">6</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/07JAN2021.xls">7</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/08JAN2021.xls">8</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/09JAN2021.xls">9</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/10JAN2021.xls">10</a>,
 <a href="https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/11JAN2021.xls">11</a

In [ ]:
for link in list_links:
    lin = link.get('href')
    if re.search('Consolidado', lin, re.IGNORECASE):
      False
    else:
      print(lin)

https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/01JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/02JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/03JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/04JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/05JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/06JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/07JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/08JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/09JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/10JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/11JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/12JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/13JAN2021.xls
https://www.prefeitura.sp.gov.br/cidade/secretarias

In [ ]:
for link in list_links: # Aqui deu certo a leitura, e todo processo é só reproduzir o mesmo que o retroativo...
    lin = link.get('href')
    if re.search(dia, lin, re.IGNORECASE):
      print(lin)

https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/28JUN2021.xls


In [ ]:
all_data = pd.DataFrame() 
for link in list_links:
    lin = link.get('href')
    if re.search(dia, lin, re.IGNORECASE):
      df1 = pd.read_excel(lin,skiprows = range(0, 2))
      all_data = all_data.append(df1)
      a= all_data.groupby(['Data'])['Passageiros Pagtes Em Dinheiro'].sum()
      b = all_data.groupby(['Data'])['Passageiros Comum e VT'].sum()
      c = all_data.groupby(['Data'])['Passageiros Pgts Bu Comum M'].sum()
      d = all_data.groupby(['Data'])['Passageiros Pagtes Estudante'].sum()
      e = all_data.groupby(['Data'])['Passageiros Pgts Bu Est Mensal'].sum()
      f = all_data.groupby(['Data'])['Passageiros Pgts Bu Vt Mensal'].sum()
      g = all_data.groupby(['Data'])['Passageiros Pagantes'].sum()
      h = all_data.groupby(['Data'])['Passageiros Int Ônibus->Ônibus'].sum()
      i = all_data.groupby(['Data'])['Passageiros Com Gratuidade'].sum()
      j = all_data.groupby(['Data'])['Passageiros Com Gratuidade Est'].sum()
      t = all_data.groupby(['Data'])['Tot Passageiros Transportados'].sum()
      i1 = pd.concat([t,a,b,c,d,e,f,g,h,i,j],axis = 1)
      i1 = i1.sort_values(by=['Data'])

In [ ]:
i1

,Tot Passageiros Transportados,Passageiros Pagtes Em Dinheiro,Passageiros Comum e VT,Passageiros Pgts Bu Comum M,Passageiros Pagtes Estudante,Passageiros Pgts Bu Est Mensal,Passageiros Pgts Bu Vt Mensal,Passageiros Pagantes,Passageiros Int Ônibus->Ônibus,Passageiros Com Gratuidade,Passageiros Com Gratuidade Est
Data,,,,,,,,,,,
2021-06-28,4444482,243478,2251585,96845,19147,50,6,2611111,1291619,493034,48718


**Aqui consegui ler sempre o dia de ontem. A ideia é ir subitraindo d -1 até que o ultimo dia salvo seja = dia**

#**Arquivos Morto**

In [ ]:
if d< 10:
  d = str(d)
  di = str("0"+d)
data_hoje = di+mes+ano
data_hoje

In [ ]:
def teste_parse(ds, **kwargs):
    r = requests.get('https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932')
    soup = BeautifulSoup(r.content, 'html.parser')
    calendario_janeiro = soup.find("caption", text = "Janeiro").parent
    dia_20 = calendario_janeiro.find("a", text = "20")
    link_do_dia_20 = dia_20['href']
    passageiros_transportados = requests.get(link_do_dia_20)
    df = pd.read_excel(passageiros_transportados.content)
    print(df)

In [ ]:
print(df)

NameError: ignored

In [ ]:
r = requests.get('https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932')
soup = BeautifulSoup(r.content, 'html.parser')

ConnectionError: ignored

In [ ]:
for str in soup2.find_all(True):
    print(str.name)

In [ ]:
dados = pd.read_excel(soup3,
                  skiprows = range(0, 2)
                     )

In [ ]:
import re
for tag in soup2.find_all(re.compile(data_hoje)):
    print(tag.name)

In [ ]:
soup_day = soup2.find('div', class_ = data_hoje)

In [ ]:
print(soup_day)

In [ ]:
link = "https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/"+data_hoje+".xls" 
link #porém, vou tentar não utilizar o link, e só a data_hoje para buscar o link no soup2.

In [ ]:
req = requests.get('https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/' + d + mes + ano + '.xls')
req

In [ ]:
data2 = pd.read_html('https://www.prefeitura.sp.gov.br/cidade/secretarias/transportes/institucional/sptrans/acesso_a_informacao/agenda/index.php?p=306932')

In [ ]:
data2 #interessante

[    Dom    Seg    Ter    Qua    Qui    Sex    Sáb
 0   NaN    NaN    NaN    NaN    NaN      1      2
 1   3.0      4      5      6      7      8      9
 2  10.0     11     12     13     14     15     16
 3  17.0     18     19     20     21     22     23
 4  24.0     25     26     27     28     29     30
 5  31.0  Total  Total  Total  Total  Total  Total,
     Dom    Seg    Ter    Qua    Qui    Sex    Sáb
 0   NaN      1      2      3      4      5      6
 1   7.0      8      9     10     11     12     13
 2  14.0     15     16     17     18     19     20
 3  21.0     22     23     24     25     26     27
 4  28.0  Total  Total  Total  Total  Total  Total,
     Dom  Seg  Ter  Qua    Qui    Sex    Sáb
 0   NaN    1    2    3      4      5      6
 1   7.0    8    9   10     11     12     13
 2  14.0   15   16   17     18     19     20
 3  21.0   22   23   24     25     26     27
 4  28.0   29   30   31  Total  Total  Total,
      Dom    Seg    Ter    Qua    Qui    Sex    Sáb
 0    NaN   